> this part is for developers who want to add features to this chatbot. This code overview will be most helpful if you have gotten the chatbot working and are now working on new features to add.

# notable frontend code:

most frontend code is just for handling the response and formatting it for display on the frontend. There is one method which you should know exists in the frontend though. However 
you won't need to make any changes likely. This function just handles sending the request to the api. Different endpoints are called by creating new methods which call this method, but pass in a different url.

In [ ]:
//in /assets/js/chatbot.js

// Function to fetch data from the API
async function fetchData(url, method = "GET", data = null) {
    const options = {
      method, // HTTP method (GET, POST, DELETE, etc.)
      headers: { "Content-Type": "application/json" }, // Headers for the request
      mode: "cors", // Cross-origin resource sharing
      cache: "no-cache", // No caching
      credentials: "same-origin", // Same-origin credentials
      redirect: "follow", // Follow redirects
      referrerPolicy: "no-referrer", // No referrer policy
    };
    if (data) options.body = JSON.stringify(data); // Add body data if provided
    const response = await fetch(url, options); // Fetch data from the API //:)
    if (!response.ok){
      const errorMsg = 'AI Bot Error: ' + response.status;
      console.log(errorMsg);
  
      return Promise.reject(errorMsg);
    }
    console.log(response); // Log the response for debugging
    return response.json(); // Return the response text
  }


See how this delete method uses this method, but passes in different parameters:

In [ ]:
async function deleteChat(id){
  const chats = await fetchData(`${urls.deleteChat}${id}?personid=${elements.personid.value}`, "DELETE"); // Send a DELETE request to clear chat history
  // it returns the update chat history
  // display updated chat history
  console.log(chats);
  elements.chat.innerHTML = ""; // Clear the chat display area
  appendMessage(assets.botName, assets.botImg, "left", "Your updated chat history!", assets.botTitle, "", formatDate(new Date())); // Inform the user that the chat history is loaded

  chats.forEach(chat => { // Loop through each chat message
    console.log(chat);
    appendMessage(assets.personName, assets.personImg, "right", chat['chatMessage'], assets.personTitle, chat['id'], formatMessageDate(chat['timestamp'])); // Append user's message
    appendMessage(assets.botName, assets.botImg, "left", chat['chatResponse'], assets.botTitle, chat['id'], formatMessageDate(chat['timestamp'])); // Append bot's response
  });
  
}

# backend code

The most important file to take note of from the chathistory folder is the AIChatbotController.java. This contains all of the endpoints and the methods associated with them. So if you wanted to add a new backend endpoint for new functionality, you would likely start here with creating the endpoint and method.

In the file itself, most of the methods are self explanatory. This method however is incredibly important, so I will go over it here:


In [ ]:
@GetMapping("/chat")
public ResponseEntity<?> chat(@RequestParam String message,@RequestParam Long personid) {
    try {
        // user sends a message that is sent to chat gpt and a response is returned
        String response = getResponseFromAI(message);
        // getResponseFromAI method is used to send actual request.
        System.out.println("Chat: " + message);
        System.out.println("Response: " + response);
        
        Chat chat = new Chat(message, response, new Date(System.currentTimeMillis()), personid);
        Chat chatUpdated = chatJpaRepository.save(chat);
        System.out.println("Chat saved in db: " + chatUpdated.getId());
        return new ResponseEntity<Chat>(chatUpdated, HttpStatus.OK);
        //return response
    } catch (Exception e) {
        e.printStackTrace();
        return new ResponseEntity<String>(e.getMessage(), HttpStatus.INTERNAL_SERVER_ERROR);
    }
}

this method is the endpoint which sends the response using the message from the frontend. Look at this function in conjunction with the frontend to understand how it works.

If you want to develop new methods, consider looking at other methods and taking inspiration from their implementations.